### 끝말잇기 게임

**멀티턴** 대화 관리 기술을 실제 게임에 적용한 예시

#### 게임 규칙
1. 이미 나온 단어를 다시 말하면 패배
2. 두음법칙 허용 (ex. 리→이, 력→역, 락→낙)
3. 국어사전에 존재하는 명사만 허용
4. **아무런 설명 없이, 끝말잇기 단어만 한글로 한 단어만 출력** (프롬프트에 명확히 지시)

#### 멀티턴 대화와의 연관성
- AI가 **이전 턴에 나온 단어들을 기억**해야 하므로, 멀티턴 대화 관리 기술이 필수적입니다.
- 대화 기록을 바탕으로 **중복 단어 방지, 규칙 위반 감지**가 가능합니다.
- 실제 서비스에서는 세션별로 기록을 분리해 여러 사용자가 동시에 게임을 즐길 수 있습니다.

#### 실습 포인트
- 프롬프트 설계가 중요: "설명 없이 단어만 출력"을 명확히 지시해야 LLM이 불필요한 설명을 하지 않음
- 대화 기록 요약/관리 기법을 게임에도 적용할 수 있음

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=gemini_api_key)

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory


# 대화 기록을 저장할 히스토리 클래스 불러오기
chat_history = ChatMessageHistory()

chat_history.messages

[]

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 끝말잇기 게임을 진행하는 AI 챗봇입니다. 아래는 게임 규칙입니다. 당신과 user 의 입력에서 아래 규칙이 꼭 지켜져야 하며, 지키지 않은 사람에게 패배를 알린 뒤, 끝말잇기 게임을 종료합니다.
                1. 주어진 대화 기록에서 이미 나왔던 단어를 다시 말했을 경우 패배합니다.
                2. 두음법칙을 허용합니다. (ex. 리 -> 이, 력 -> 역, 락 -> 낙)
                3. 국어사전에 존재하는 단어이자, 명사여야 합니다.
                4. 아무런 설명 없이, 끝말잇기 단어만 한글로 한 단어만 출력하세요.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | llm

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [7]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "위 채팅 메시지는 끝말잇기 게임을 진행한 대화내용입니다. 언급한 단어들만 나열하여 저장해주세요.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | llm

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [ ]:
while(True):
    user_input = input("🧑 YOUR TURN : ")
    if user_input == "종료": break
    response = chain_with_summarization.invoke(
                {"input": user_input},
                {"configurable": {"session_id": "unused"}},
            )
    print("🤖 AI TURN : ", response.content)

🤖 AI TURN :  면도
🤖 AI TURN :  마차
🤖 AI TURN :  지우개
